In [1]:
import cv2
import numpy as np
import math

In [2]:
v1_f1 = cv2.imread("OneStopNoEnter1cor0249.bmp",0)
v1_f2 = cv2.imread("OneStopNoEnter1cor0251.bmp",0)

### Find the motion compensated difference image 
#### (Video frame ) mdf1 = determineMotionVector(f1,f2)

In [3]:
def blockMatching(img1,img2,blockSize,searchSize):
    searchSize = (int)((searchSize - blockSize)/2)
    newImg = np.zeros((len(img1),len(img1[0])))
    motionImage = np.zeros((len(img1),len(img1[0])))
    for i in range(0,min(len(img1),len(img2)-1),blockSize):
        for j in range(0,min(len(img1[0]),len(img2[0])-1),blockSize):
            block1 = sumOfBlocks(img1,i,j,blockSize)
            block2 = sumOfBlocks(img2,i,j,blockSize)
            difference=[]
            points = []
            if(block1-block2 !=0):
                for x in range(-searchSize,(searchSize+blockSize)):
                    for y in range(-searchSize,(searchSize+blockSize)):
                        block2 = sumOfBlocks(img2,i+x,j+y,blockSize)
                        difference.append(abs((block1)- block2))
                        points.append((i+x,j+y))
                if(difference == []):
                    pixel=0
                else:
                    temp = findMinimum(difference,points)
                    pixel = temp[0]
                    u = temp[1][0]
                    v = temp[1][1]
                    for m,x in zip(range(u,u+blockSize),range(i,i+blockSize)):
                        for n,y in zip(range(v,v+blockSize),range(j,j+blockSize)):
                            if(m<len(img1) and n<len(img1[0])):
                                newImg[m][n] = pixel
                                newImg[x][y] = pixel
                           
    return newImg

In [4]:
def findMinimum(difference,points):
    min = 2048888
    for i in range(0,len(difference)):
        if(min> difference[i]):
            min=difference[i]
            point=points[i]
    #print(min)
    return [min,point]

In [5]:
def sumOfBlocks(img,x,y,blockSize):
    sum_ = 0
    for j in range(0,blockSize):
        for i in range(0,blockSize):
            if((x+j < len(img)) and (y+i < len(img[0]))):
                sum_ += img[x+j][y+i]
    return sum_

In [6]:
mdf1 = np.array(blockMatching(v1_f1,v1_f2,8,16))
cv2.imwrite("mdf1.jpg",mdf1)

True

### Question4 Find the DCT transformed images
##### DCT_mdf1 = DCT(mdf1)


In [7]:
def applyDCT(img,size,noOfPixel):
    newImg = np.zeros((len(img),len(img[0])))
    DCTMatrix = createDCTMatrix(size)
    DCTMatrixTranspose = np.transpose(DCTMatrix)
    for i in range(0,len(img),size):
        for j in range(0,len(img[0]),size):
            block = createBlocks(img,i,j,size)
            DCTOutput = (np.matmul(DCTMatrixTranspose,np.matmul(block,DCTMatrix)))
            for x,u in zip(range(i,i+size),DCTOutput):
                for y,v in zip(range(j,j+size),u):
                    newImg[x][y] = v
    return newImg

In [8]:
def createBlocks(img,x,y,blockSize):
    blocks=[]
    block=[]
    for j in range(0,blockSize):
        xblock=[]
        for i in range(0,blockSize):
            if((x+j < len(img)) and (y+i < len(img[0]))):
                xblock.append(img[x+j][y+i])
        block.append(xblock)
    blocks.append(block)
    return block

In [9]:
def createDCTMatrix(size):
    matrix = np.zeros((size,size))
    for i in range(0,len(matrix)):
        for j in range(0,len(matrix[0])):
            if(i == 0):
                matrix[i][j] = (1/size) ** 0.5
            else:
                term = ((2*j)+1)*math.pi*i
                term = term/(2*size)
                matrix[i][j] = (((2/size) **0.5) * math.cos(term))
    return matrix

In [10]:
DCT_mdf1 = applyDCT(mdf1,8,4)

# Apply IDCT

In [11]:
def applyIDCT(img,size):
    newImg = np.zeros((len(img),len(img[0])))
    DCTMatrix = createDCTMatrix(size)
    DCTMatrixTranspose = np.transpose(DCTMatrix)
    for i in range(0,len(img),size):
        for j in range(0,len(img[0]),size):
            block = createBlocks(img,i,j,size)
            IDCTOutput = (np.matmul(DCTMatrix,np.matmul(block,DCTMatrixTranspose)))
            for x,u in zip(range(i,i+size),IDCTOutput):
                for y,v in zip(range(j,j+size),u):
                        newImg[x][y] = v
    return newImg

In [12]:
IDCT_mdf1 = applyIDCT(DCT_mdf1,8)
cv2.imwrite("IDCT_mdf1.jpg",IDCT_mdf1)

True

#Reconstruct the Image
##### f2' = f1 + IDCT_df1

In [13]:
IDCT_df1 = cv2.imread("IDCT_mdf1.jpg",0)

In [14]:
def add(img1,img2):
    newImg = np.zeros((len(img1),len(img1[0])))
    for i in range(0,min(len(img2),len(img1))):
        for j in range(0,min(len(img2[0]),len(img1[0]))):
            addition = (img1[i][j] + img2[i][j])
            if(addition > 0 ):
                newImg[i][j] = addition
            else:
                newImg[i][j] = 0
    return newImg

In [15]:
f2_ = add(v1_f1,IDCT_mdf1)
cv2.imwrite("f2'.jpg",f2_)

True